# Implemenation of spelling correction solutions for locations in proGres v4

In [ ]:
'''
The find_substitute function demo can be used to suggest location name corrections
based on list of official location names 
using the soundex and doublemetaphone algorithms

Section A: consists of a prototype using sample data
Section B: includes the following steps:
           - 1) import correct/official location level names from proGres v4 and OCHA lists for Egypt (COA) and Syria (COO)
           - 2) import location level fields from proGres v4 (i.e. progres_placeofbirthcity & COO/COA location names (123) 
           - 3) identify locations (list 2) that are not included in official list of locations (list 1) at L1, L2, or L3
           - cleaning steps: including filtering out entries that are too short and/or in Arabic script
           - call find_substitute function to find alternate spellings for list 3 comparing the spellings against list 1
           
Next Steps: optimize implementation by adapting the function for pandas series: pd.Series.apply(lambda x: find_substitute(x))
'''

# A. DEMO with sample data

## A.1. Notebook setup

#### Import libraries needed

In [62]:
import numpy as np
import pandas as pd
import pickle
import datetime as datetime
import pyodbc
import urllib
import sqlalchemy
import requests
import time

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 0)

from metaphone import doublemetaphone
from langdetect import detect, DetectorFactory
DetectorFactory.seed = 0
import re
import fuzzy

## A.2. Function to find substitutes

In [63]:
def find_substitute(list_of_cities, correct_list, algorithm=doublemetaphone):
    
    # Get code for each city originally listed
    city_list = []
    code_city_list = []
    for city in list_of_cities:
        if city!=None:
            orig_city = city.encode('ascii', 'ignore').decode('ascii')
            if algorithm == "soundex":
                alg = fuzzy.Soundex(4)
                alg_city = alg(city.encode('ascii', 'ignore').decode('ascii'))
            elif algorithm == "metaphone":
                alg = doublemetaphone
                alg_city = alg(city.encode('ascii', 'ignore').decode('ascii'))
            else:
                alg = doublemetaphone
                alg_city = alg(city)
            city_list.append(orig_city)
            code_city_list.append(alg_city)
        else:
            city_list.append(city)
            code_city_list.append(city)
            
    dict_city_code = dict(zip(city_list, code_city_list))
    
    # Get code for each correct city at the relevant level
    correct_city_list = []
    code_correct_city_list = []
    for city in correct_list:
        if city!=None:
            city = re.sub(r'[!@#$(),\n"%^*?\:;~`\d]', '', city)
            orig_city = city.encode('ascii', 'ignore').decode('ascii')
            if algorithm == "soundex":
                alg = fuzzy.Soundex(4)
                alg_city = alg(city.encode('ascii', 'ignore').decode('ascii'))
            elif algorithm == "metaphone":
                alg = doublemetaphone
                alg_city = alg(city)
            else:
                alg = doublemetaphone
                alg_city = alg(city.encode('ascii', 'ignore').decode('ascii'))
            correct_city_list.append(orig_city)
            code_correct_city_list.append(alg_city)
        else:
            correct_city_list.append(city)
            code_correct_city_list.append(city)
    
    dict_correct_code_city = dict(zip(code_correct_city_list, correct_city_list))

    # print outputs directly
#     # Match city name to potential correct city
#     for city in city_list:
#         if dict_city_code[city] in code_correct_city_list:
#             print(city, dict_correct_code_city[dict_city_code[city]])
#         else:
#             print(city, "no match found")
        
    # Print outputs as a matched dataframe    
    # Match city name to potential correct city
    df_output_all = pd.DataFrame()
    for city in city_list:
        df_output = pd.DataFrame()
        if dict_city_code[city] in code_correct_city_list:
            df_output.loc[1,"v4_name"] = city
            df_output.loc[1,"v4_name_suggested"] = dict_correct_code_city[dict_city_code[city]]
        else:
            df_output.loc[1,"v4_name"] = city
            df_output.loc[1,"v4_name_suggested"] = "No match found"
        df_output_all = pd.concat([df_output_all, df_output])
    return df_output_all

#### Sample birth cities and correct location names

In [65]:
birth_cities = ["qalubya", "faisal", "beni suef" , "qalubiya", "qaliobya", "qalupia", "bollakk el dakror"]
# birth_cities = dfi.progres_birthcityname.tolist()

ocha_v4_cities = ["Faysal", "Qalyub", "Bani Swayf City", "Bulaq Al-DakrUr"]
# ocha_v4_cities =  admin3_name_ocha

In [64]:
### Sample 1 with metaphone

In [66]:
find_substitute(birth_cities,
                ocha_v4_cities,
               algorithm = "metaphone")

,v4_name,v4_name_suggested
1,qalubya,Qalyub
1,faisal,Faysal
1,beni suef,No match found
1,qalubiya,Qalyub
1,qaliobya,Qalyub
1,qalupia,Qalyub
1,bollakk el dakror,No match found


In [67]:
### Sample 2 with soundex

In [69]:
find_substitute(birth_cities,  
                ocha_v4_cities, 
                "soundex")

,v4_name,v4_name_suggested
1,qalubya,Qalyub
1,faisal,Faysal
1,beni suef,Bani Swayf City
1,qalubiya,Qalyub
1,qaliobya,Qalyub
1,qalupia,Qalyub
1,bollakk el dakror,Bulaq Al-DakrUr


## END OF DEMO

# B. Implementation using location data from proGres v4 

### i.e. Get locations from Egypt and Syria and check for possible spelling corrections

## B.1. Get list of location names from proGres

### Import proGres v4 Individual and Location Level Tables

#### Connect to database

In [229]:
# text1 = ''
# text2 = ''
saveFile = open('pwd.txt', 'r+')
# saveFile.write(text1 + ',' + text2)
# saveFile.seek(0)
uap = saveFile.read()

In [230]:
ip_of_server = "**.***.**.**"

driver = 'SQL Server'
ip = ip_of_server
db_connection = pyodbc.connect(
    driver = driver,
    Server = ip,
    Port = "1433",
    Database = "QualityCheck_Egypt",
    UID = uap.split(',')[0],
    PWD = uap.split(',')[1])

In [231]:
# checking
pyodbc.drivers()

['SQL Server']

In [232]:
# checking
type(db_connection)

pyodbc.Connection

#### Import v4 Individual table

In [233]:
dfi = pd.read_sql_query("SELECT * FROM dbo.Filteredprogres_individual", db_connection)
dfi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290220 entries, 0 to 290219
Data columns (total 55 columns):
 #   Column                             Non-Null Count   Dtype         
---  ------                             --------------   -----         
 0   progres_refugeestatus              290220 non-null  int64         
 1   progres_educationlevelname         288184 non-null  object        
 2   progres_coolocationlevel3name      70561 non-null   object        
 3   progres_nationalitylookupname      290213 non-null  object        
 4   progres_registrationreason         290220 non-null  int64         
 5   progres_countryofasylumidname      290220 non-null  object        
 6   progres_id                         290220 non-null  object        
 7   progres_fathersname                285626 non-null  object        
 8   progres_registrationdate           290220 non-null  datetime64[ns]
 9   progres_givenname                  290215 non-null  object        
 10  createdbyname       

#### Filter for active and hold and Business Unit in MENA

In [234]:
dfi[["statuscode" , "statuscodename", "progres_individualid"]].groupby(["statuscode" , "statuscodename"], dropna=False).count()

,,progres_individualid
statuscode,statuscodename,
1,Active,290178
125080000,Hold,42


In [235]:
# Filter for statuscode [1, 125080000] 
# which correspond to statuscodename [Active, Hold] respectively
dfi = dfi[dfi.statuscode.isin([1, 125080000])]
dfi.shape

(290220, 55)

In [236]:
# Only keep MENA business units

In [237]:
list_mena_bu = ['Mauritania - Bassikounou',
                'Mauritania - Urban',
                'Tunisia - CO',
                'Egypt - CO',
                'Lebanon - North',
                'Lebanon - BML',
                'Lebanon - South',
                'Lebanon - Bekaa',
                'Algeria - CO',
                'Iraq - CO',
                'Israel - CO',
                'Jordan - Camps',
                'Jordan - Urban',
                'Kuwait - CO',
                'Morocco - CO',
                'RO Riyadh',
                'Syria - CO',
                'UAE - CO']

In [238]:
# dfi[["progres_businessunit" , 
#      "progres_businessunitname", 
#      "progres_individualid"]].groupby(["progres_businessunit" , 
#                                        "progres_businessunitname"], dropna=False).count()

In [239]:
dfi = dfi[dfi.progres_businessunitname.isin(list_mena_bu)]
dfi.shape

(290220, 55)

#### Check for and remove any duplicated entries

In [240]:
dfi[dfi.duplicated(keep=False)].head()

,progres_refugeestatus,progres_educationlevelname,progres_coolocationlevel3name,progres_nationalitylookupname,progres_registrationreason,progres_countryofasylumidname,progres_id,progres_fathersname,progres_registrationdate,progres_givenname,createdbyname,progres_religionidname,progres_arrivaldate,progres_dateofbirth,progres_registrationreasonname,progres_coalocationlevel2name,progres_coolocationlevel1name,progres_hasphoto,progres_agecohortname,progres_placeofbirthcity,progres_refugeestatusname,progres_countryoforiginidname,progres_countryofasylumid,statuscodename,createdon,progres_businessunitname,progres_coalocationlevel1name,progres_nationalitylookup,progres_businessunit,progres_indvidualid,progres_maritalstatusname,progres_registrationgroupidname,progres_sexname,modifiedon,progres_refugeestatuscategoryname,progres_primaryphonenumber,progres_placeofbirthidname,progres_maritalstatus,progres_relationshiptofpname,statuscode,progres_registrationgroupid,progres_biometricstatus,progres_deceaseddate,progres_fleddate,progres_isdeceased,progres_sex,progres_relationshiptofp,modifiedbyname,progres_individualid,progres_age,progres_coalocationlevel3name,progres_coolocationlevel2name,progres_familyname,progres_countryoforiginid,progres_ethnicityidname


#### Import v4 combined ocha v4 data

In [16]:
coa_addresses = pd.read_sql_query("SELECT * FROM dbo.Egypt_locations",
                                  db_connection)

In [17]:
coa_addresses.shape

(6216, 17)

In [18]:

coo_addresses = pd.read_sql_query("SELECT * FROM dbo.Syria_locations",
                                  db_connection)

In [19]:
coo_addresses.shape

(5746, 17)

## B.2. Get list of official location names and location names in proGres

In [20]:
coa_matched = coa_addresses[
                            (~coa_addresses.admin1_pcode_ocha.isna())  # & # if null, this pcode exists in v4 but not in OCHA
                            # (~coa_addresses.progres_locationlevel.isna())  # if null, this pcode exists in OCHA but not in v4 ( can comment out to include these if we're checking the levels also)
                            ]
coo_matched = coo_addresses[
                            (~coo_addresses.admin1_pcode_ocha.isna())  # & # if null, this pcode exists in v4 but not in OCHA
                            # (~coo_addresses.len_pcode_v1.isna())  # if null, this pcode exists in v4 but not in OCHA ( cam comment out to include this if we're checking the levels also)
                            ]

In [21]:
coo_matched.head()

,progres_countryidname,progres_locationlevel,progres_level,progres_code,ocha_v4_pcode,admin3_name_ocha,admin3_name_ar_ocha,admin3_pcode_ocha,admin2_name_ocha,admin2_name_ar_ocha,admin2_pcode_ocha,admin1_name_ocha,admin1_name_ar_ocha,admin1_pcode_ocha,admin0_name_ocha,admin0_name_ar_ocha,admi0_pcode_ocha
0,Syrian Arab Republic,Damascus,1.0,21SYR001,SY01,None,None,None,None,None,None,Damascus,????,SY01,Syrian Arab Republic,????????? ??????? ???????,SY
1,Syrian Arab Republic,Aleppo,1.0,21SYR002,SY02,None,None,None,None,None,None,Aleppo,???,SY02,Syrian Arab Republic,????????? ??????? ???????,SY
2,Syrian Arab Republic,Rural Damascus,1.0,21SYR003,SY03,None,None,None,None,None,None,Rural Damascus,??? ????,SY03,Syrian Arab Republic,????????? ??????? ???????,SY
3,Syrian Arab Republic,Homs,1.0,21SYR004,SY04,None,None,None,None,None,None,Homs,???,SY04,Syrian Arab Republic,????????? ??????? ???????,SY
4,Syrian Arab Republic,Hama,1.0,21SYR005,SY05,None,None,None,None,None,None,Hama,????,SY05,Syrian Arab Republic,????????? ??????? ???????,SY


In [22]:

coa_names_1 = coa_matched[coa_matched.progres_level == 1].progres_locationlevel.unique().tolist()
coa_names_1_ocha = coa_matched[coa_matched.progres_level == 1].admin1_name_ocha.unique().tolist()

coa_names_2 = coa_matched[coa_matched.progres_level == 2].progres_locationlevel.unique().tolist()
coa_names_2_ocha = coa_matched[coa_matched.progres_level == 2].admin2_name_ocha.unique().tolist()

coa_names_3 = coa_matched[coa_matched.progres_level == 3].progres_locationlevel.unique().tolist()
coa_names_3_ocha = coa_matched[coa_matched.progres_level == 3].admin3_name_ocha.unique().tolist()

coo_names_1 = coo_matched[coo_matched.progres_level == 1].progres_locationlevel.unique().tolist()
coo_names_1_ocha = coo_matched[coo_matched.progres_level == 1].admin1_name_ocha.unique().tolist()

coo_names_2 = coo_matched[coo_matched.progres_level == 2].progres_locationlevel.unique().tolist()
coo_names_2_ocha = coo_matched[coo_matched.progres_level == 2].admin2_name_ocha.unique().tolist()

coo_names_3 = coo_matched[coo_matched.progres_level == 3].progres_locationlevel.unique().tolist()
coo_names_3_ocha = coo_matched[coo_matched.progres_level == 3].admin3_name_ocha.unique().tolist()

In [508]:
all_egypt_locations = list(set(coa_names_1 +
                               coa_names_1_ocha + 
                               coa_names_2 +
                               coa_names_2_ocha + 
                               coa_names_3 +
                               coa_names_3_ocha))

all_syria_locations = list(set(coo_names_1 +
                               coo_names_1_ocha + 
                               coo_names_2 +
                               coo_names_2_ocha + 
                               coo_names_3 +
                               coo_names_3_ocha))

In [23]:
coa_names_3_ocha

['<Null>']

In [249]:
print(len(coo_names_1_ocha), len(coo_names_2_ocha), len(coo_names_3_ocha))

14 62 271


In [250]:
print(len(coo_names_1), len(coo_names_2), len(coo_names_3))

14 62 346


In [251]:
# level 3 in english didn't exist in ocha list
print(len(coa_names_1_ocha), len(coa_names_2_ocha), len(coa_names_3_ocha))

27 331 1


In [252]:
print(len(coa_names_1), len(coa_names_2), len(coa_names_3))

27 356 786


#### Syria L1 L2 L3

In [291]:
dfisy = dfi[(dfi.progres_countryoforiginidname=="Syrian Arab Republic") \
            # &
           # (dfi.progres_relationshiptofpname=="Focal Point")
           ]
dfisy.shape

(144714, 55)

In [295]:
def classify_as_unknown(df_to_check_for_unknowns):
    '''
    irregularity if focal point progres location level names don't match 
    1) the names in OCHA 
    OR 
    2) the names in the location level table in proGres v4 (matched against ocha pcodes)
    '''
    
    df = df_to_check_for_unknowns.copy()
    
    # including progres names
    df.loc[:,"syr_l1"] = df.progres_coolocationlevel1name.apply(lambda x: "Unknown" if x not in list(set(coo_names_1 + coo_names_1_ocha)) else x)
    df.loc[:,"syr_l2"] = df.progres_coolocationlevel2name.apply(lambda x: "Unknown" if x not in list(set(coo_names_2 + coo_names_2_ocha)) else x)
    df.loc[:,"syr_l3"] = df.progres_coolocationlevel3name.apply(lambda x: "Unknown" if x not in list(set(coo_names_3 + coo_names_3_ocha)) else x)
    
    # # excluding progres names
    # df.loc[:,"syr_l1"] = df.progres_coolocationlevel1name.apply(lambda x: "Unknown" if x not in list(set(coo_names_1_ocha)) else x)
    # df.loc[:,"syr_l2"] = df.progres_coolocationlevel2name.apply(lambda x: "Unknown" if x not in list(set(coo_names_2_ocha)) else x)
    # df.loc[:,"syr_l3"] = df.progres_coolocationlevel3name.apply(lambda x: "Unknown" if x not in list(set(coo_names_3_ocha)) else x)
    
    return df


In [296]:
dfisy = classify_as_unknown(dfisy)
dfisy.shape

(144714, 58)

In [300]:
syr_level1= dfisy[(dfisy.syr_l1=="Unknown")&(dfisy.progres_coolocationlevel1name!=None)].progres_coolocationlevel1name.unique().tolist()

In [301]:
syr_level2= dfisy[(dfisy.syr_l2=="Unknown")&(dfisy.progres_coolocationlevel2name!=None)].progres_coolocationlevel2name.unique().tolist()

In [302]:
syr_level3= dfisy[(dfisy.syr_l3=="Unknown")&(dfisy.progres_coolocationlevel3name!=None)].progres_coolocationlevel3name.unique().tolist()

In [303]:
print(len(syr_level1), len(syr_level2), len(syr_level3))

11 5 45


#### Egypt L1 L2 L3

In [304]:
def coa_classify_as_unknown(df):
    '''
    irregularity if focal point progres location level names don't match 
    1) either the names in OCHA OR 
    2) the names in the location level table in proGres v4 (matched against ocha pcodes)
    '''
    # including progres v4 names
    df.loc[:,"coa_l1"] = df.progres_coalocationlevel1name.apply(lambda x: "Unknown" if x not in list(set(coa_names_1 + coa_names_1_ocha)) or x==None else x)
    df.loc[:,"coa_l2"] = df.progres_coalocationlevel2name.apply(lambda x: "Unknown" if x not in list(set(coa_names_2 + coa_names_2_ocha)) or x==None else x)
    df.loc[:,"coa_l3"] = df.progres_coalocationlevel3name.apply(lambda x: "Unknown" if x not in list(set(coa_names_3)) or x==None else x)
    # for egypt remove ocha names at level 3 because ocha didn't include these names
    
    # excluding progres v4 names 
    # df.loc[:,"coa_l1"] = df.progres_coalocationlevel1name.apply(lambda x: "Unknown" if x not in list(set(coa_names_1_ocha)) or x==None else x)
    # df.loc[:,"coa_l2"] = df.progres_coalocationlevel2name.apply(lambda x: "Unknown" if x not in list(set(coa_names_2_ocha)) or x==None else x)
    # df.loc[:,"coa_l3"] = df.progres_coalocationlevel3name.apply(lambda x: "Unknown" if x not in list(set(coa_names_3)) or x==None else x)
   
    return df

In [305]:
dfi = coa_classify_as_unknown(dfi)
dfi.shape

(290220, 58)

In [319]:
egy_level1= dfi[(dfi.coa_l1=="Unknown")&(dfi.progres_coalocationlevel1name!=None)].progres_coalocationlevel1name.unique().tolist()

In [320]:
egy_level2= dfi[(dfi.coa_l2=="Unknown")&(dfi.progres_coalocationlevel2name!=None)].progres_coalocationlevel2name.unique().tolist()

In [321]:
egy_level3= dfi[(dfi.coa_l3=="Unknown")&(dfi.progres_coalocationlevel3name!=None)].progres_coalocationlevel3name.unique().tolist()

#### Syria Birth Cities

In [277]:
birth_country_name = "Syrian Arab Republic"
sy = dfi[(dfi.progres_placeofbirthidname==birth_country_name)&
         (~dfi.progres_placeofbirthcity.isna())]

In [278]:
sy["birth_city_length"] = sy["progres_placeofbirthcity"].apply(lambda x: len(x) if x!=None else None)
sy = sy[sy.birth_city_length>1]

C:\Users\RIJAL\Anaconda3\envs\phonenumber\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [280]:
sy[sy.birth_city_length==26].progres_placeofbirthcity

4490    Latakia, Al-Hanadi Village
Name: progres_placeofbirthcity, dtype: object

In [281]:
sy[sy.birth_city_length<=3].progres_placeofbirthcity.unique()

array(['syr', 'UAE', 'Dam', 'Ale', 'dam', 'Hma', 'KSA', 'DAM', 'حمص',
       'AL', 'SYR', 'HOS', 'sy', 'hom', 'ale', 'ksa', 'HAM', 'Bab', 'HOM',
       'ARE', 'Da', 'حلب', 'dan'], dtype=object)

In [282]:
sy["birth_city_language"] = sy["progres_placeofbirthcity"].apply(lambda x: detect(x) if x!=None else None) 

In [283]:
# Filter out birth cities listed in Arabic
sy = sy[sy.birth_city_language!="ar"]

In [284]:
sy["birth_city"] = sy["progres_placeofbirthcity"].apply(lambda x: x.lower())
sy["birth_city"] = sy["birth_city"].apply(lambda x: re.sub(r'[!@#$(),\n"%^*?\:;~`\d]', '', x) if x!=None else None)

In [ ]:
sy.progres_placeofbirthcity.value_counts()[0:10]

In [ ]:
sy.birth_city.value_counts()[0:10]

In [287]:
orig_list_of_birth_cities = sy.birth_city.unique().tolist()

In [289]:
len(orig_list_of_birth_cities)

1160

In [288]:
city_len = []
for city in orig_list_of_birth_cities:
    city_len.append(len(city))
list(set(city_len))

[2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 25,
 26]

#### Egypt Birth Cities

In [259]:
detect("السويس")

'ar'

In [260]:
detect("6 اكتوبر")

'ar'

In [261]:
eg = dfi[(dfi.progres_placeofbirthidname=="Egypt")&
         (~dfi.progres_placeofbirthcity.isna())]

In [262]:
eg["birth_city_length"] = eg["progres_placeofbirthcity"].apply(lambda x: len(x) if x!=None else None)
eg = eg[eg.birth_city_length>1]

C:\Users\RIJAL\Anaconda3\envs\phonenumber\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [264]:
eg[eg.birth_city_length==34].progres_placeofbirthcity

184457    Cairo -Saqr kurash - madenit Nasr 
Name: progres_placeofbirthcity, dtype: object

In [265]:
eg[eg.birth_city_length<=3].progres_placeofbirthcity.unique()

array(['Dam', 'Egy', 'Aga', 'EGY', 'are', 'SUD', 'EGP', 'egy', 'مصر',
       'ARE', 'dam', 'AGA', 'Giz', 'DAM'], dtype=object)

In [266]:
eg["birth_city_language"] = eg["progres_placeofbirthcity"].apply(lambda x: detect(x) if x!=None else None) 

In [267]:
# Filter out birth cities listed in Arabic
eg = eg[eg.birth_city_language!="ar"]

In [270]:
eg["birth_city"] = eg["progres_placeofbirthcity"].apply(lambda x: x.lower())
eg["birth_city"] = eg["birth_city"].apply(lambda x: re.sub(r'[!@#$(),\n"%^*?\:;~`\d]', '', x) if x!=None else None)

In [ ]:
eg.progres_placeofbirthcity.value_counts()[0:10]

In [ ]:
eg.birth_city.value_counts()[0:10]

In [273]:
egypt_birth_cities = eg.birth_city.unique().tolist()

In [274]:
# egypt_birth_cities.remove('/')
# egypt_birth_cities.remove('مصر')

## B.3. Call the find_substitute function on Egypt/Syria list of locations against official list

In [509]:
#For example: 
metaphone_egypt_correct_birth_ocha1 = find_substitute(egypt_birth_cities, coa_names_1_ocha, "metaphone")

In [518]:
soundex_egypt_correct_birth_ocha1 = find_substitute(egypt_birth_cities, coa_names_1_ocha, "soundex")